# Data Processing

Qidu Fu

**Contents**
- [0 Import libraries](#0_import_libraries)
- [1 Load data](#1-load-data)
    - [1.1 Get stack exchange data](#11-get-stack-exchange-data)
    - [1.2 Get SEAME data](#12-get-seame-data-development-dataset)
    - [1.3 Get ASCEND data](#13-get-ascend-data)
- [2 Process the stack exchange data: the title column](#2-process-the-stack-exchange-data-the-title-column)
- [3 Process the stack exchange data: the tags and title column](#3-process-the-stack-exchange-tags-and-title-columns)
- [4 Get basic information: cleaned stack exchange dataset](#4-get-basic-information-cleaned-stack-exchange-dataset)

## 0 Import libraries



In [49]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ssl
import datasets
import re

## 1 Load data

### 1.1 Get stack exchange data
Observations
- 1. Not NULL values in the data
- 2. The dataset include 12401 records/rows and 3 columns

In [50]:
# The stack exchange data is from stack exchange API

def get_stack_data(path):
    """
    Load the stack exchange data
    param path: str: path to the stack exchange data
    return: pd.DataFrame: the stack exchange data
    """
    return pd.read_csv(path)

STACK_DF = get_stack_data('private/stack_exchange_all_questions.csv')
print(STACK_DF.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12041 entries, 0 to 12040
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   question_id  12041 non-null  int64 
 1   title        12041 non-null  object
 2   tags         12041 non-null  object
dtypes: int64(1), object(2)
memory usage: 282.3+ KB
None


In [51]:
STACK_DF.head()

,question_id,title,tags
0,59815,My translation of Li Bai&#39;s 《三五七言》,"translation, poetry"
1,53654,What do these characters on an antique mural p...,"character-identification, traditional-characte..."
2,59823,Help in translating Li Bai&#39;s 《月下独酌&#183;其二》,"translation, poetry"
3,59821,purpose of using 了 with 要不,grammar
4,59652,Why does the character 的 is pronounced differe...,"pronunciation, songs"


### 1.2 Get SEAME data (development dataset)
Observations
- 1. Not NULL values in the data
- 2. The dataset include 5321 records/rows and 3 columns

In [52]:
# The SEAME DATASET IS FROM: 
# Zhiping Zeng, Yerbolat Khassanov, Van Tung Pham, Haihua Xu, Eng Siong Chng, 
# and Haizhou Li, “On the End-to-End Solution to Mandarin-English Code-switching 
# Speech Recognition,” arXiv preprint arXiv:1811.00241, 2018.

def get_seame_data(path):
    """
    This function reads the SEAME dataset from the given path 
    and returns a pandas dataframe.
    param path: str: path to the SEAME dataset
    return: pd.DataFrame: SEAME dataset
    """
    # Ignore SSL certificate errors
    ssl._create_default_https_context = ssl._create_unverified_context
    # Read the file without specifying column names
    df = pd.read_csv(path, sep='\t', header=None)
    # Reset SSL certificate to default
    ssl._create_default_https_context = ssl._create_default_https_context
    # Split the first column into 'ID' and 'sentence'
    df[['ID', 'sentence']] = df[0].str.split(pat=' ', n=1, expand=True)
    # Drop the original column
    df = df.drop(columns=[0])
    return df

SEAME_DF = get_seame_data('https://raw.githubusercontent.com/zengzp0912/' + 
                            'SEAME-dev-set/refs/heads/master/dev_sge/text')

In [53]:
SEAME_DF.head()

,ID,sentence
0,nc15m-08nc15mbp_0101-00190-00481,hello hello 可 以
1,nc15m-08nc15mbp_0101-01044-01130,往 下 一 点
2,nc15m-08nc15mbp_0101-02185-02444,那 个 wave 比 较 慢 一 点 啊
3,nc15m-08nc15mbp_0101-02503-02571,okay 可 以
4,nc15m-08nc15mbp_0101-02838-02963,讲 多 一 点 咯 可 以


In [54]:
SEAME_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5321 entries, 0 to 5320
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        5321 non-null   object
 1   sentence  5321 non-null   object
dtypes: object(2)
memory usage: 83.3+ KB


In [55]:
# save it to a csv file for future use
SEAME_DF.to_csv('private/seame.csv', index=False)

### 1.3 Get ASCEND data
Observations
- 1. Not NULL values in the data
- 2. The dataset include 9869 records/rows and 3 columns

In [56]:
# The third dataset is ASCEND
# The ASCEND DATASET IS FROM:
# Lovenia, H., Cahyawijaya, S., Winata, G., Xu, P., Xu, Y., Liu, Z., Frieske, 
# R., Yu, T., Dai, W., Barezi, E. J., Chen, Q., Ma, X., Shi, B., & Fung, 
# P. (2022, June). ASCEND: A spontaneous Chinese-English dataset 
# for code-switching in multi-turn conversation. Proceedings of the 
# Language Resources and Evaluation Conference, 7259–7268. European 
# Language Resources Association. https://aclanthology.org/2022.lrec-1.788

def get_ascend_data():
    """
    Load the ASCEND dataset
    param: None
    return: pd.DataFrame: the ASCEND training and development datasets
    """
    # Load dataset but remove audio feature
    data = datasets.load_dataset('CAiRE/ascend')

    # Convert dataset to a Pandas DataFrame
    ascend_tr_df = data['train'].to_pandas()
    # Use the development set for now
    ascend_dev_df = data['validation'].to_pandas()

    # Keep only the required columns
    ascend_tr_df = ascend_tr_df[["id", "transcription", "topic"]]
    ascend_dev_df = ascend_dev_df[["id", "transcription", "topic"]]

    return ascend_tr_df, ascend_dev_df
# Load ASCEND dataset with selected columns
ASCEND_DF, ASCEND_DV_DF = get_ascend_data()
ASCEND_DV_DF.head()


,id,transcription,topic
0,00000,嗯,technology
1,00001,小朋友我回想一下when i was in elementary school,technology
2,00002,like year three,technology
3,00003,i have a phone but it's not a smart phone at t...,technology
4,00004,就是其实主要的功能就是打电话然后跟parents联系,technology


In [57]:
# ASCEND_DV_DF.info()

In [58]:
ASCEND_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9869 entries, 0 to 9868
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             9869 non-null   object
 1   transcription  9869 non-null   object
 2   topic          9869 non-null   object
dtypes: object(3)
memory usage: 231.4+ KB


In [59]:
# save it to a csv file for future use
ASCEND_DF.to_csv('private/ascend.csv', index=False)

## 2 Process the stack exchange data: the title column
Data processing pipeline aims at cleaning to the dataset to the level of the 
SEAME dataset from the published Chinese-English code-switching [paper](https://arxiv.org/abs/1811.00241). 
- 1. Randomly sample 20 rows/records multiple times for uncleanliness checking
- 2. Check URLs (non present)
    - URLs are not relevant to the code-switching. 
- 3. Replace special symbols/html encoding for their actual characters, such as &quot as "
    - These special symbols are not HTML representations of the actual characters. 
    The replacement correct that.
- 4. Remove non-Chinese-or-English characters, such as number+letters, 
    letters+numbers
    - This project focuses on Chinese-English code-switching. 
    Non-Chinese-or-English characters are irrelevant, which may be gibberish
    from website scraping. 

In [60]:
def sample_rows(df, n=20):
    """
    This function samples n rows from the given dataframe.
    param df: pd.DataFrame: dataframe to sample from
    param n: int: number of rows to sample
    return: pd.DataFrame: sampled dataframe
    """
    return df.sample(n)

sample_rows(STACK_DF)

,question_id,title,tags
5942,16157,How do I translate the first two lines of Befr...,translation
3193,45500,When to use classifier,measure-word
11060,9141,the meaning of 算什么 in 《我爱台妹》,"grammar, taiwan, lyrics"
11044,9217,How to express this idea in a better way?,"word-choice, phrase, equivalent-phrase"
8423,25966,What&#39;s this chinese symbol mean?,meaning
9133,22817,Question about how to say &quot;sounds like&qu...,"translation, grammar, vocabulary"
6021,27177,Does 以安倍的自民党为首的联盟 use the 以 ~ 为 pattern?,"meaning-in-context, phrase"
10434,13561,Is 未長未短 grammatically correct (in classical Ch...,"grammar, vocabulary, usage, classical-chinese"
11973,428,Etymology of 他妈的,etymology
11475,5969,Are there words in Mandarin written by charact...,"mandarin, writing"


In [61]:
# Function to check all the rows with a given pattern
def check_pattern_non_regex(df, pattern):
    """
    Check all the rows with a given pattern
    param df: pd.DataFrame: dataframe to check
    param pattern: str: pattern to check
    return: pd.DataFrame: dataframe with rows containing the pattern
    """
    return df[df.str.contains(pattern, na=False)]

def check_pattern_regex(df, pattern):
    """
    Check all the rows with a given pattern
    param df: pd.DataFrame: dataframe to check
    param pattern: str: pattern to check
    return: pd.DataFrame: dataframe with rows containing the pattern
    """
    return df[df.str.contains(pattern, na=False, regex=True)]

In [62]:
# Check if url is present the stack exchange data
check_pattern_regex(STACK_DF['title'], r'https?://\S+')
# This will return all the rows with a url in the title column

Series([], Name: title, dtype: object)

In [63]:

# Function to replace A with B using regex
def replace_text(text, pattern, replace_with):
    """
    Replace a pattern in a text with a given string
    param text: str: input text
    param pattern: str: pattern to replace
    param replace_with: str: string to replace the pattern with
    return: str: text with the pattern replaced
    """
    return re.sub(pattern, replace_with, text)

# Function to replace " &quot;" with '"' and "&#39;" with "'"
def replace_quot(text):
    """
    Replace &quot; with "
    param text: str: input text
    return: str: text with &quot or &#39;; replaced with "
    """
    text = replace_text(text, "&quot;", '"')
    text = replace_text(text, "&#39;", "'")
    return text

# Test the function
print(replace_quot('This is a &quot;test&quot; string'))

This is a "test" string


In [64]:
# Apply the function to the title quotes in the stack exchange dataset
STACK_DF['title'] = STACK_DF['title'].apply(replace_quot)
print(STACK_DF.iloc[10710, 1])
print(STACK_DF.iloc[11633, 1])
print(STACK_DF.iloc[2547, 1])

What is the difference: "和小猫一样可爱" and "像小猫一样可爱"
How do you say 'dates back to' in Mandarin?
一 [份/个/项] 工作??? What is the difference?


In [65]:
print(STACK_DF.iloc[8067]['title'])
print(STACK_DF.iloc[4462]['title'])

Why 批 means "criticize" and "a batch" at the same time?
What's the exact difference between 住 and 居?


In [66]:
# Function to remove 's, 'll, and others
def remove_apostrophe(text):
    """
    Remove 's, 'll, 've, 're, 'd, 'm, 'em from text
    param text: str: input text
    return: str: text with 's, 'll, 've, 're, 'd, 'm, 'em removed
    """
    text = re.sub(r"'ll", "", text)
    text = re.sub(r"'ve", "", text)
    text = re.sub(r"'re", "", text)
    text = re.sub(r"'d", "", text)
    text = re.sub(r"'m", "", text)
    text = re.sub(r"'em", "", text)
    text = re.sub(r"'t", "", text)
    text = re.sub(r"n't", "not", text)
    text = re.sub(r"'", "", text)
    text = re.sub(r'"s', "", text)
    text = re.sub(r'"ll', "", text)
    text = re.sub(r'"ve', "", text)
    text = re.sub(r'"re', "", text)
    text = re.sub(r'"d', "", text)
    text = re.sub(r'"m', "", text)
    text = re.sub(r'"em', "", text)
    text = re.sub(r'"t', "", text)
    text = re.sub(r'"', "", text)
    text = re.sub(r"’s", "", text)
    text = re.sub(r"’ll", "", text)
    text = re.sub(r"’ve", "", text)
    text = re.sub(r"’re", "", text)
    text = re.sub(r"’d", "", text)
    text = re.sub(r"’m", "", text)
    text = re.sub(r"’em", "", text)
    text = re.sub(r"’t", "", text)
    text = re.sub(r"n’t", "not", text)
    text = re.sub(r"’", "", text)
    text = re.sub(r"‘s", "", text)
    text = re.sub(r"‘ll", "", text)
    text = re.sub(r"‘ve", "", text)
    text = re.sub(r"‘re", "", text)
    text = re.sub(r"‘d", "", text)
    text = re.sub(r"‘m", "", text)
    text = re.sub(r"‘em", "", text)
    text = re.sub(r"‘t", "", text)
    text = re.sub(r"n‘t", "not", text)
    text = re.sub(r"‘", "", text)
    return text

# remove_apostrophe("I'm going to the store. He'll be there.")
# remove_apostrophe(STACK_DF.iloc[192, 1])

In [67]:
print(STACK_DF.iloc[192, 1])
# STACK_DF['title'] = STACK_DF['title'].apply(remove_apostrophe)
print(STACK_DF.iloc[192, 1])

Understanding and interpretation of 李白's 《玉阶怨》
Understanding and interpretation of 李白's 《玉阶怨》


In [68]:
# Resample again to check other signs of data uncleanliness
sample_rows(STACK_DF)

,question_id,title,tags
4229,22290,Why is hot water called 开水?,vocabulary
11070,9071,Can you 关心 an object?,grammar
11147,1130,"Is 再过［时间］ the most common way of expressing ""i...",sentence-structure
833,56390,How to incorporate a new region into Chinese c...,"characters, linguistics"
754,56654,Is 恁 related to 您?,"topolect, character-variant"
1096,55515,What about all the possible combinations of to...,"tones, listening"
2217,51675,"Chinese version of ""beat around the bush""","translation, idioms"
11866,2415,"What is a ""Babu cream""?",translation
4722,35078,What’s the characters on this seal of the vase...,character-identification
2240,50034,I would like to know the translation of the wr...,"translation, character-identification, seal"


In [69]:
print('STACK_DF.iloc[7598, 1]: ')
print(STACK_DF.iloc[7598, 1])
print('STACK_DF.iloc[4374, 1]: ')
print(STACK_DF.iloc[4374, 1])
print('STACK_DF.iloc[1041, 1]: ')
print(STACK_DF.iloc[1041, 1])
print('STACK_DF.iloc[1230, 1]: ')
print(STACK_DF.iloc[1230, 1])
print('STACK_DF.iloc[6224, 1]: ')
print(STACK_DF.iloc[6224, 1])
print('STACK_DF.iloc[664, 1]: ')
print(STACK_DF.iloc[664, 1])
print('STACK_DF.iloc[5366, 1]: ')
print(STACK_DF.iloc[5366, 1])

STACK_DF.iloc[7598, 1]: 
Is this [Ivanka Trump] Chinese proverb real? —“Those who say it can not be done, should not interrupt those doing it”
STACK_DF.iloc[4374, 1]: 
What semantic notions underlie "to think; to contemplate", with "but; however; nevertheless" with "only"?
STACK_DF.iloc[1041, 1]: 
Should the blank in 他毕业以后很快 _____ 找到了一个理想的工作 be filled with 才, 可, or 便?
STACK_DF.iloc[1230, 1]: 
Can you say someone is a ”非常熟的人“？
STACK_DF.iloc[6224, 1]: 
“管你是[surname] + [name]还是[same surname] + [different name]”
STACK_DF.iloc[664, 1]: 
D&#224;o d&#233; and d&#224;od&#233;. Difference between each word separately? And when combined together as a compound word?
STACK_DF.iloc[5366, 1]: 
好客	vs 亲切 - both mean hospitable?


In [70]:
# replace D&#224;o d&#233; and d&#224;od&#233 with correct terms
STACK_DF['title'] = STACK_DF['title'].apply(replace_text, args=('D&#224;o d&#233;', 'Dao de'))
STACK_DF['title'] = STACK_DF['title'].apply(replace_text, args=('d&#224;od&#233;', 'daode'))
STACK_DF['title'] = STACK_DF['title'].apply(replace_text, args=('d&#233;', 'de'))
print(STACK_DF.iloc[664, 1])
print(STACK_DF.iloc[2907, 1])

Dao de and daode. Difference between each word separately? And when combined together as a compound word?
Why is 肯德基 (Kěndejī) = Kentucky Fried Chicken not called 肯德鸡 (Kěndejī) when it's pronounced the same and 鸡 means "chicken"?


In [71]:
# Check underscores in the title column
# check_pattern_regex(STACK_DF['title'], r'_+')

In [72]:
# remove underscores from the title column
# STACK_DF['title'] = STACK_DF['title'].apply(lambda x: re.sub(r'_+', ' ', x))
# check_pattern_regex(STACK_DF['title'], r'_+')

In [73]:
# Resample again to check other signs of data uncleanliness
sample_rows(STACK_DF)

,question_id,title,tags
10489,13305,"Formality of ""悲哀"" as opposed to ""悲傷"" and ""哀傷""","word-choice, meaning, usage, difference, formal"
5757,36766,What do these four phrases mean in English?,translation
3862,14727,Email as noun and verb,"word-choice, vocabulary, loanwords"
4077,5807,"""醡"" and ""炸"" in ""醡醬麵"" and ""炸醬麵""","word-choice, traditional-vs-simplified, food, ..."
1487,54214,What does 敦 mean in 敦伦?,word
5031,39552,Why is 的 mandatory in the sentence 坐四十五分钟的二百七路,"grammar, syntax"
482,58517,How to pronounce 那邊?,pronunciation
4389,40913,"How to say ""can/can’t use"" in Chinese?","translation, mandarin"
6265,35421,Does (western) humor exist in Chinese culture?...,"culture, humor"
1413,54453,Comprehension of role(s) of 也 which is used in...,grammar


In [74]:
import html

# Define the function to decode all HTML and Unicode encodings
def decode_html_entities(text):
    # Step 1: Decode named and decimal numeric entities (e.g., &#224;, &#xE0;)
    text = html.unescape(text)
    
    # Step 2: Decode hexadecimal numeric entities (e.g., &#xE0;)
    text = re.sub(r'&#x([0-9A-Fa-f]+);', lambda x: chr(int(x.group(1), 16)), text)
    
    # Step 3: Decode decimal numeric entities (e.g., &#224;)
    text = re.sub(r'&#(\d+);', lambda x: chr(int(x.group(1))), text)

    # Step 4: Decode Unicode escaped sequences like \uXXXX manually
    text = re.sub(r'\\u([0-9A-Fa-f]{4})', lambda x: chr(int(x.group(1), 16)), text)

    return text

# Apply the decoding function to the 'title' column
STACK_DF['title'] = STACK_DF['title'].apply(decode_html_entities)

# Apply the decoding function to the 'tags' column if needed
STACK_DF['tags'] = STACK_DF['tags'].apply(decode_html_entities)


In [75]:
print('STACK_DF.iloc[7488, 1]: ')
print(STACK_DF.iloc[7488, 1])
print('STACK_DF.iloc[9375, 1]: ')
print(STACK_DF.iloc[9375, 1])
print('STACK_DF.iloc[6587, 1]: ')
print(STACK_DF.iloc[6587, 1])
print('STACK_DF.iloc[335, 1]: ')
print(STACK_DF.iloc[335, 1])

STACK_DF.iloc[7488, 1]: 
Why is 把 in 我去洗把脸 (wǒ qù xǐ bǎ liǎn) = I'm going to wash my face?
STACK_DF.iloc[9375, 1]: 
Blessed bat pun: 蝙蝠→__福
STACK_DF.iloc[6587, 1]: 
Why is 弄 (nòng) in 他终于弄明白了?
STACK_DF.iloc[335, 1]: 
How did Middle Chinese "hæwk" evolve into Mandarin "xué"?


In [76]:
check_pattern_non_regex(STACK_DF['title'], 'q249')

Series([], Name: title, dtype: object)

In [77]:
# remove numbers/words with numbers from the title column
print(STACK_DF.iloc[3638, 1])
# STACK_DF['title'] = STACK_DF['title'].apply(lambda x: re.sub(r'\w*\d\w*', '', x))
print(STACK_DF.iloc[3638, 1])
print(check_pattern_non_regex(STACK_DF['title'], 'q249'))

Is there any distinction between 二百 (èrbǎi) and 两百 (liǎng bǎi) which both mean "two hundred"?
Is there any distinction between 二百 (èrbǎi) and 两百 (liǎng bǎi) which both mean "two hundred"?
Series([], Name: title, dtype: object)


In [78]:
# Function to remove non English or Chinese characters
def remove_non_en_ch(text):
    """
    Remove non-English or Chinese characters
    param text: str: input text
    return: str: text with non-English or Chinese characters removed
    """
    return re.sub(r'[^\x00-\x7F\u4E00-\u9FA5]', '', text)

# print(remove_non_en_ch(STACK_DF.iloc[11613, 1]))

In [79]:
print(STACK_DF.iloc[11613, 1])
# STACK_DF['title'] = STACK_DF['title'].apply(remove_non_en_ch)
print(STACK_DF.iloc[11613, 1])

Is 去 (qù) pronunced /tɕʰu/ or /tɕʰy/?
Is 去 (qù) pronunced /tɕʰu/ or /tɕʰy/?


In [80]:
# Resample again to check other signs of data uncleanliness
sample_rows(STACK_DF)

,question_id,title,tags
11880,2306,Grammar for counting in Chinese (for non-nativ...,"grammar, number"
9185,22525,How do I look up a graph in 說文通訓定聲?,"classical-chinese, dictionary, radicals"
10123,14166,Is it appropriate to use 喂 by text?,"word-choice, usage, loanwords"
4432,34699,Fonts that compose Ideographic Description Cha...,"characters, software, fonts"
11716,3480,Chinese proverb: If you want happiness for a l...,translation
7586,1612,What is the top part of 姜 and why does it differ?,"characters, radicals"
7281,31529,How did the abbreviations 520 and 521 arise?,"etymology, internet-slang"
6956,29857,Questions on a Hakka song and its particular H...,"pronunciation, topolect, spelling, hakka"
4893,39862,Is there an ancient chinese name for Asia?,"history, place-names"
3528,43367,What's the difference between a han character ...,etymology


In [81]:
# check to remove rows with null values on the title column
STACK_DF = STACK_DF.dropna(subset=['title'])
STACK_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12041 entries, 0 to 12040
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   question_id  12041 non-null  int64 
 1   title        12041 non-null  object
 2   tags         12041 non-null  object
dtypes: int64(1), object(2)
memory usage: 282.3+ KB


## 3 Process the stack exchange: tags and title columns
Data processing pipeline aims at cleaning to the dataset to the level of the 
SEAME dataset from the published Chinese-English code-switching [paper](https://arxiv.org/abs/1811.00241). 
The tags columns are much cleaners as compared to the previous column
- Remove empty strings in the title column and replace with NaN values
- Remove rows with NaN values in the title column
- Rename the column to be `text` and `topic`
- Save the stack exchange data to a CSV to avoid repeatition processing

In [82]:
# check for non English or Chinese  characters
check_pattern_non_regex(STACK_DF['tags'], r'[^\x00-\x7F\u4E00-\u9FA5]')

Series([], Name: tags, dtype: object)

In [83]:
# check for punctuations if other than ,
check_pattern_non_regex(STACK_DF['tags'], r'[^\w\s,]')

1        character-identification, traditional-characte...
5        translation, character-identification, seal, c...
11                               classical-chinese, poetry
13             translation, character-identification, seal
17        translation, word-choice, terminology, loanwords
                               ...                        
12023                               word-choice, cantonese
12026                                          word-choice
12031                            verbs, meaning-in-context
12032                             mandarin, mainland-china
12038                 characters, character-identification
Name: tags, Length: 4342, dtype: object

In [84]:
# remove - from the tags columns
# STACK_DF['tags'] = STACK_DF['tags'].apply(lambda x: re.sub(r'-', ' ', x))
check_pattern_non_regex(STACK_DF['tags'], r'-')

1        character-identification, traditional-characte...
5        translation, character-identification, seal, c...
11                               classical-chinese, poetry
13             translation, character-identification, seal
17        translation, word-choice, terminology, loanwords
                               ...                        
12023                               word-choice, cantonese
12026                                          word-choice
12031                            verbs, meaning-in-context
12032                             mandarin, mainland-china
12038                 characters, character-identification
Name: tags, Length: 4342, dtype: object

In [85]:
sample_rows(STACK_DF)

,question_id,title,tags
945,56021,How to literally translate this: 会因为费用增加而让投资人不高兴?,"translation, meaning, meaning-in-context"
2373,50704,What is the semantic extension of 成？,"etymology, semantics, semantic-extensions"
7724,29614,剩一点儿都没地方搁 How is the sentence composed of?,"grammar, usage"
11525,5640,How to translate 实时新闻定制和推送系统 to English?,"translation, word-choice, word"
7269,1743,Is the description for components of 条 correct?,"simplified-characters, character-identification"
9031,23530,"For 錢應該花在刀口上, 錢 and 刀口 are so different things...",idioms
5664,29796,What is the difference between 都没有 and 不都有?,grammar
188,58355,Help to identify artist and the message (Chara...,"translation, character-identification, seal, c..."
5051,39497,"What is the difference between 走路, 行走, and 步行?",word-choice
11523,5706,"Translation of 我想会吗? - I thought, is it possible?",translation


In [86]:
print('STACK_DF.iloc[4763, 1]: ')
print(STACK_DF.iloc[4763, 2])
# replace - with space in the tags column
STACK_DF['tags'] = STACK_DF['tags'].apply(lambda x: re.sub(r'-', ' ', x))
print(STACK_DF.iloc[4763, 2])

STACK_DF.iloc[4763, 1]: 
traditional-characters, software, fonts
traditional characters, software, fonts


In [87]:
# rename the columns, and only select the title and tags columns
STACK_DF = STACK_DF.rename(columns={'title': 'text', 'tags': 'topic'})[['text', 'topic']]
STACK_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12041 entries, 0 to 12040
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    12041 non-null  object
 1   topic   12041 non-null  object
dtypes: object(2)
memory usage: 188.3+ KB


In [88]:
# remove empty strings from the text column
print(STACK_DF.iloc[4957, ])
STACK_DF['text'] = STACK_DF['text'].apply(lambda x: x.strip())
print(STACK_DF.iloc[4957, ])
# convert empty strings to NaN
STACK_DF['text'] = STACK_DF['text'].replace('', np.nan)
STACK_DF.info()

text     HSK6的第101题（缩写）请求反馈：富有爸爸和会花钱的儿子
topic             writing critique, hsk
Name: 4957, dtype: object
text     HSK6的第101题（缩写）请求反馈：富有爸爸和会花钱的儿子
topic             writing critique, hsk
Name: 4957, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12041 entries, 0 to 12040
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    12041 non-null  object
 1   topic   12041 non-null  object
dtypes: object(2)
memory usage: 188.3+ KB


In [89]:
# drop NaN values if title is empty
STACK_DF = STACK_DF.dropna(subset=['text'])
STACK_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12041 entries, 0 to 12040
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    12041 non-null  object
 1   topic   12041 non-null  object
dtypes: object(2)
memory usage: 188.3+ KB


In [90]:
# save the cleaned df to a csv file
STACK_DF.to_csv('private/stack_exchange_cleaned.csv', index=False)
print('STACK_DF saved to private/stack_exchange_cleaned.csv')

STACK_DF saved to private/stack_exchange_cleaned.csv


In [91]:
# save the first 1000 rows of the cleaned df to a csv file for sample 
STACK_DF.head(1000).to_csv('data_samples/stack_exchange_cleaned_sample.csv', index=False)
print('First 1000 rows of STACK_DF saved to data_samples/stack_exchange_cleaned_sample.csv')

First 1000 rows of STACK_DF saved to data_samples/stack_exchange_cleaned_sample.csv


## 4 Get basic information: cleaned stack exchange dataset

The cleaned dataset contains **12,039 entries** with two columns:

- **text**: Posts from Stack Exchange (12,039 non-null values, `object` string).
- **topic**: General discourse domain of each post (12,039 non-null values, `object` string).

Additional information
- The `topic` column still needs processing, such as word tokenization or others 
for topic modeling/clustering. At the moment, it has 3500 combinations using the
`value_counts` function.
- The text column may still needs further processing, such as word tokenization 
and others. 

In [92]:
CLEANED_STACK_DF = pd.read_csv('private/stack_exchange_cleaned.csv')
CLEANED_STACK_DF.head()

,text,topic
0,My translation of Li Bai's 《三五七言》,"translation, poetry"
1,What do these characters on an antique mural p...,"character identification, traditional characte..."
2,Help in translating Li Bai's 《月下独酌·其二》,"translation, poetry"
3,purpose of using 了 with 要不,grammar
4,Why does the character 的 is pronounced differe...,"pronunciation, songs"


In [93]:
CLEANED_STACK_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12041 entries, 0 to 12040
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    12041 non-null  object
 1   topic   12041 non-null  object
dtypes: object(2)
memory usage: 188.3+ KB


In [94]:
CLEANED_STACK_DF['topic'].value_counts()

translation                                                            974
grammar                                                                601
meaning                                                                464
meaning in context                                                     238
word choice                                                            217
                                                                      ... 
terms of address, formal                                                 1
meaning in context, etymology, character identification                  1
idioms, colloquialisms                                                   1
character identification, traditional characters, seal, old chinese      1
translation, style                                                       1
Name: topic, Length: 3502, dtype: int64